In [1]:
import os
import torch
import probtorch
from utils.file_handling import load_nparray
from models.architectures.encoders.fc import HFVAEFCEncoder
from models.architectures.decoders.fc import HFVAEFCDecoder
from models.hfvae import HFVAE
from multiprocessing import cpu_count
from trainer import VAETrainer
import time

os.chdir('..')
print(os.getcwd())
# model parameters
HIDDEN_DIM = 256
LATENT_DIM = 50

# training parameters
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
BETA1 = 0.90
EPS = 1e-9
CUDA = torch.cuda.is_available()

# saving
SAVE_PATH = 'results/hfvae'


/home/dorian/FER/master_thesis/src/base


In [ ]:
kwargs = {'num_workers': cpu_count(), 'pin_memory': True} if CUDA else {'num_workers': cpu_count()}
train_loader = torch.utils.data.DataLoader(load_nparray('resources/datasets/20_newsgroups/preprocessed/train_bow.txt'),
                                         batch_size=BATCH_SIZE, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(load_nparray('resources/datasets/20_newsgroups/preprocessed/test_bow.txt'),
                                        batch_size=BATCH_SIZE, shuffle=True, **kwargs)


enc = HFVAEFCEncoder(input_dim=2000, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM)
dec = HFVAEFCDecoder(latent_dim=LATENT_DIM, output_dim=2000, batch_size=BATCH_SIZE)
model = HFVAE(encoder=enc, decoder=dec)
model.cuda()
model.double()
optimizer =  torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
device = torch.device("cuda" if CUDA else "cpu")

VAETrainer(model, device, train_loader, test_loader, save_model_path=SAVE_PATH, probtorch=True).run(optimizer, NUM_EPOCHS)


Train Epoch:  1 [     0/ 11314 (  0%)]	Loss: 2.800763
Train Epoch:  1 [  1280/ 11314 ( 11%)]	Loss: 2.024806
Train Epoch:  1 [  2560/ 11314 ( 22%)]	Loss: 2.875154
Train Epoch:  1 [  3840/ 11314 ( 34%)]	Loss: 1.744349
Train Epoch:  1 [  5120/ 11314 ( 45%)]	Loss: 3.015428
Train Epoch:  1 [  6400/ 11314 ( 56%)]	Loss: 2.833321
Train Epoch:  1 [  7680/ 11314 ( 67%)]	Loss: 2.775266
Train Epoch:  1 [  8960/ 11314 ( 79%)]	Loss: 2.348330
Train Epoch:  1 [ 10240/ 11314 ( 90%)]	Loss: 1.751881
====> Epoch: 1 Average loss: 2.5703
====> Test set loss: 2.3904
Train Epoch:  2 [     0/ 11314 (  0%)]	Loss: 4.840169
Train Epoch:  2 [  1280/ 11314 ( 11%)]	Loss: 2.256938
Train Epoch:  2 [  2560/ 11314 ( 22%)]	Loss: 3.681406
Train Epoch:  2 [  3840/ 11314 ( 34%)]	Loss: 2.250000
Train Epoch:  2 [  5120/ 11314 ( 45%)]	Loss: 2.412110
Train Epoch:  2 [  6400/ 11314 ( 56%)]	Loss: 1.918739
Train Epoch:  2 [  7680/ 11314 ( 67%)]	Loss: 4.262239
Train Epoch:  2 [  8960/ 11314 ( 79%)]	Loss: 2.830435
Train Epoch:  2 [ 

In [ ]:
# EVALUATION
# model loading
# model = HFVAE()
# model.load_state_dics(torch.load('results/hfvae/model.pt'))
# model.eval()

# Embedding visualization



# t-sne
from sklearn.manifold import TSNE





In [ ]:
# BACKUP TRAINER
# train_loader = torch.utils.data.DataLoader(load_nparray('resources/datasets/20_newsgroups/preprocessed/train_bow.txt'),
#                                          batch_size=BATCH_SIZE, shuffle=True)
# test_loader = torch.utils.data.DataLoader(load_nparray('resources/datasets/20_newsgroups/preprocessed/test_bow.txt'),
#                                         batch_size=BATCH_SIZE, shuffle=True)
#
#
# enc = HFVAEFCEncoder(input_dim=2000, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM)
# dec = HFVAEFCDecoder(latent_dim=LATENT_DIM, output_dim=2000)
# model = HFVAE(encoder=enc, decoder=dec)
# model.cuda()
# model.double()
# optimizer =  torch.optim.Adam(list(model.parameters()), lr=LEARNING_RATE, betas=(BETA1, 0.999))
#
# def train(data, model, optimizer):
#     epoch_elbo = 0.0
#     model.train()
#     N = 0
#     for b, texts in enumerate(data):
#         if texts.size()[0] == BATCH_SIZE:
#             N += BATCH_SIZE
#             if CUDA:
#                 texts = texts.cuda()
#             optimizer.zero_grad()
#             q, p = model(texts)
#             loss = model.loss_function(q, p)
#             loss.backward()
#             optimizer.step()
#             if CUDA:
#                 loss = loss.cpu()
#             epoch_elbo -= float(loss.item())
#     return epoch_elbo / N
#
# def test(data, model):
#     model.eval()
#     epoch_elbo = 0.0
#     N = 0
#     for b, texts in enumerate(data):
#         if texts.size()[0] == BATCH_SIZE:
#             N += BATCH_SIZE
#             if CUDA:
#                 texts = texts.cuda()
#             q, p = model.forward(texts)
#             batch_elbo = -model.loss_function(q, p)
#             if CUDA:
#                 batch_elbo = batch_elbo.cpu()
#             epoch_elbo += float(batch_elbo.item())
#     return epoch_elbo / N
#
# for e in range(NUM_EPOCHS):
#     train_start = time.time()
#     train_elbo = train(train_loader, model, optimizer)
#     train_end = time.time()
#     test_start = time.time()
#     test_elbo = test(test_loader, model)
#     test_end = time.time()
#     print('[Epoch %d] Train: ELBO %.4e (%ds) Test: ELBO %.4e (%ds)' % (
#             e, train_elbo, train_end - train_start,
#             test_elbo, test_end - test_start))